In [ ]:
import asyncio
from typing import List

from genlm.backend import load_model_by_name
from genlm.bytes import BeamParams
from genlm.control import ByteLLM, AWRS, direct_token_sampler
from genlm.eval.core import run_evaluation
from genlm.eval.core.model import ModelOutput, ModelResponse
from genlm.eval.domains.spider.spider import (
    SpiderDataset,
    SpiderEvaluator,
    default_prompt_formatter,
)
from genlm.eval.domains.spider.table_column_potential import SpiderTableColumnVerifier
from pprint import pprint as pp

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# HF_TOKEN should be set via environment variable or huggingface-cli login
# "/teamspace/studios/this_studio/spider_data"
SPIDER_SAMPLE_DIR = "/teamspace/studios/this_studio/genlm-eval/assets/spider/spider_sample"
SPIDER_DATA_DIR = "/teamspace/studios/this_studio/spider_data"
SPIDER_GRAMMARS = "/teamspace/studios/this_studio/genlm-eval/assets/spider/grammars.json"


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 09-24 19:05:07 [__init__.py:235] Automatically detected platform cuda.


In [21]:
def build_bytelm():
    llm = load_model_by_name("meta-llama/Llama-3.2-1B-Instruct", backend="hf")
    # llm = load_model_by_name("gpt2", backend="hf")
    model_eos_token = llm.byte_vocab[llm.tokenizer.eos_token_id]
    beam_params = BeamParams(
        K=1,
        prune_threshold=0.0,
        eos_tokens=[b"\n", b"\n\n", b"<|eot_id|>"],
        heal=True,
    )
    return ByteLLM(llm, beam_params)

In [22]:
BYTE_LLM = build_bytelm()
dataset = SpiderDataset.from_spider_dir(
        SPIDER_DATA_DIR,
        grammar_json_path=SPIDER_GRAMMARS,
        few_shot_example_ids=[],
    )
sampleset = SpiderDataset.from_spider_dir(
        SPIDER_SAMPLE_DIR,
        grammar_json_path=SPIDER_GRAMMARS,
        few_shot_example_ids=[],
    )


In [23]:
from itertools import islice

def show_first_two(ds, label):
    items = list(islice(ds, 2))
    for i, inst in enumerate(items):
        print(f"[{label}] #{i} schema={inst.schema_name}")
        print(f"utterance: {inst.utterance}")
        print(f"gold: {inst.gold}\n")

show_first_two(dataset, "full")
show_first_two(sampleset, "sample")

[full] #0 schema=concert_singer
utterance: How many singers do we have?
gold: SELECT count(*) FROM singer

[full] #1 schema=concert_singer
utterance: What is the total number of singers?
gold: SELECT count(*) FROM singer

[sample] #0 schema=concert_singer
utterance: How many singers do we have?
gold: SELECT count(*) FROM singer

[sample] #1 schema=concert_singer
utterance: What is the total number of singers?
gold: SELECT count(*) FROM singer



In [24]:
first_full = next(iter(dataset))
first_sample = next(iter(sampleset))

print(first_full.schema_name, first_full.utterance, first_full.gold)
print(first_sample.schema_name, first_sample.utterance, first_sample.gold)

full_prompt_ids = default_prompt_formatter(
        BYTE_LLM.llm.tokenizer,
        first_full,
        use_chat_format=False,
    )
sample_prompt_ids = default_prompt_formatter(
        BYTE_LLM.llm.tokenizer,
        first_sample,
        use_chat_format=False,
    )
print("full prompt: ", BYTE_LLM.llm.tokenizer.decode(full_prompt_ids))
print("*"*80)
print("sample prompt: ", BYTE_LLM.llm.tokenizer.decode(sample_prompt_ids))


concert_singer How many singers do we have? SELECT count(*) FROM singer
concert_singer How many singers do we have? SELECT count(*) FROM singer
full prompt:  <|begin_of_text|>You are a coding assistant helping an analyst answer questions over business data in SQL. More specifically, the analyst provides you a database schema (tables in the database along with their column names and types) and asks a question about the data that can be solved by issuing a SQL query to the database. In response, you write the SQL statement that answers the question. You do not provide any commentary or explanation of what the code does, just the SQL statement ending in a semicolon.



Here is a database schema:
stadium
* Stadium_ID (number): stadium id
* Location (text): location
* Name (text): name
* Capacity (number): capacity
* Highest (number): highest
* Lowest (number): lowest
* Average (number): average

singer
* Singer_ID (number): singer id
* Name (text): name
* Country (text): country
* Song_Nam

In [25]:
BYTE_LLM.set_prompt_from_str(BYTE_LLM.llm.tokenizer.decode(full_prompt_ids))
condition = SpiderTableColumnVerifier(grammar=first_full.lark_grammar, tables=first_full.tables).coerce(BYTE_LLM, f=b"".join)
sampler = AWRS(BYTE_LLM, condition)
sequences = await sampler.smc(
        n_particles=5,
        ess_threshold=0.9,
        max_tokens=70,
        verbosity=1,    
        )

print("Decoded posterior:", sequences.decoded_posterior)



Task was destroyed but it is pending!
task: <Task cancelling name='Task-6762' coro=<AsyncTokenByteTrie._background_loop() running at /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/genlm/bytes/trie.py:570> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-6763' coro=<AsyncTokenByteTrie._background_loop() running at /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/genlm/bytes/trie.py:570> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-6764' coro=<AsyncTokenByteTrie._background_loop() running at /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/genlm/bytes/trie.py:570> wait_for=<Future cancelled>>
Task was destroyed but it is pending!
task: <Task cancelling name='Task-6765' coro=<AsyncTokenByteTrie._background_loop() running at /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/genlm/bytes/trie.py:570> wait_for=<Fut

0.00:	[\n]
0.00:	[\n]
0.00:	[\n]
0.00:	[␣]
0.00:	[␣]
0.00:	[\n|\n]
0.00:	[\n|`]
0.00:	[\n|\n]
0.00:	[␣|␣]
0.00:	[␣|\n]
0.00:	[\n|\n|#]
0.00:	[\n|\n|#]
0.00:	[\n|`|`]
0.00:	[␣|␣|;]
0.00:	[␣|\n|1]
0.00:	[\n|\n|#|␣]
0.00:	[\n|\n|#|#]
0.00:	[\n|`|`|`]
0.00:	[␣|␣|;|␣]
0.00:	[␣|\n|1|0]
0.00:	[\n|\n|#|␣|C]
0.00:	[\n|`|`|`|s]
0.00:	[\n|\n|#|#|#]
0.00:	[␣|␣|;|␣|S]
-0.19:	[␣|\n|1|0|␣]
0.00:	[\n|\n|#|␣|C|r]
0.00:	[\n|`|`|`|s|q]
0.00:	[\n|\n|#|#|#|␣]
0.00:	[␣|␣|;|␣|S|E]
-0.19:	[␣|\n|1|0|␣|7]
0.00:	[\n|\n|#|␣|C|r|e]
0.00:	[\n|`|`|`|s|q|l]
0.00:	[\n|\n|#|#|#|␣|R]
0.00:	[␣|␣|;|␣|S|E|L]
-0.19:	[␣|\n|1|0|␣|7|␣]


/teamspace/studios/this_studio/genlm-eval/genlm/eval/domains/spider/table_column_potential.py:88: UserWarning: Failed to parse complete  
10
  warnings.warn(f"Failed to parse complete {context}")


0.00:	[\n|\n|#|␣|C|r|e|a]
-3.69:	[\n|`|`|`|s|q|l|␣]
0.00:	[\n|\n|#|#|#|␣|R|a]
0.00:	[␣|␣|;|␣|S|E|L|E]
-0.19:	[␣|\n|1|0|␣|7|␣|A]
-0.26:	[\n|\n|#|#|#|␣|R|a|t]
-0.26:	[\n|\n|#|␣|C|r|e|a|t]
-0.26:	[\n|\n|#|␣|C|r|e|a|t]
-0.26:	[\n|`|`|`|s|q|l|␣|\n]
-0.26:	[\n|`|`|`|s|q|l|␣|\n]
-0.26:	[\n|\n|#|␣|C|r|e|a|t|e]
-0.26:	[\n|\n|#|␣|C|r|e|a|t|e]
-0.26:	[\n|\n|#|#|#|␣|R|a|t|i]
-0.26:	[\n|`|`|`|s|q|l|␣|\n|S]
-0.26:	[\n|`|`|`|s|q|l|␣|\n|S]
-0.26:	[\n|\n|#|#|#|␣|R|a|t|i|n]
-0.26:	[\n|\n|#|␣|C|r|e|a|t|e|␣]
-0.26:	[\n|\n|#|␣|C|r|e|a|t|e|␣]
-0.26:	[\n|`|`|`|s|q|l|␣|\n|S|E]
-0.26:	[\n|`|`|`|s|q|l|␣|\n|S|E]
-0.26:	[\n|\n|#|#|#|␣|R|a|t|i|n|g]


/teamspace/studios/this_studio/genlm-eval/genlm/eval/domains/spider/table_column_potential.py:88: UserWarning: Failed to parse complete 
```sql
  warnings.warn(f"Failed to parse complete {context}")


...

-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|s|u|m|␣|o|f|␣|s|i|n|g|e|r|s]
-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|t|a|b|l|e|␣|a|l|i|a|s|e|s|␣|f|o|r|␣|s|t|a|d|i|u|m|,|␣|s]
-0.49:	[\n|`|`|`|s|q|l|␣|\n|S|E|L|E|C|T|␣|C|O|U|N|T|(|*|)|␣|F|R|O|M|␣|s|i|n|g|e|r|_|i|n]
-1.25:	[\n|`|`|`|s|q|l|␣|\n|S|E|L|E|C|T|␣|C|O|U|N|T|(|*|)|␣|F|R|O|M|␣|s|i|n|g|e|r|␣|I|N]
-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e]
-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|t|a|b|l|e|␣|a|l|i|a|s|e|s|␣|f|o|r|␣|s|t|a|d|i|u|m|,|␣|s|i]
-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|s|u|m|␣|o|f|␣|s|i|n|g|e|r|s|␣]
-0.49:	[\n|`|`|`|s|q|l|␣|\n|S|E|L|E|C|T|␣|C|O|U|N|T|(|*|)|␣|F|R|O|M|␣|s|i|n|g|e|r|_|i|n|_]
-1.25:	[\n|`|`|`|s|q|l|␣|\n|S|E|L|E|C|T|␣|C|O|U|N|T|(|*|)|␣|F|R|O|M|␣|s|i|n|g|e|r|␣|I|N|␣]
-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r]
-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s]
-0.49:	

/teamspace/studios/this_studio/genlm-eval/genlm/eval/domains/spider/table_column_potential.py:88: UserWarning: Failed to parse complete 
```sql 
SELECT COUNT(*) FROM singer_in_concert
  warnings.warn(f"Failed to parse complete {context}")
/teamspace/studios/this_studio/genlm-eval/genlm/eval/domains/spider/table_column_potential.py:88: UserWarning: Failed to parse complete 

# Create an aggregated sum of singers by location
  warnings.warn(f"Failed to parse complete {context}")


-1.31:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|s|u|m|␣|o|f|␣|s|i|n|g|e|r|s|␣|b|y|␣|l|o|c|a|t|i|o|n|␣]
-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣]
-0.49:	[\n|\n|#|␣|C|r|e|a|t|e|␣|t|a|b|l|e|␣|a|l|i|a|s|e|s|␣|f|o|r|␣|s|t|a|d|i|u|m|,|␣|s|i|n|g|e|r|,|␣|c|o|n|c|e|r]
-0.71:	[\n|`|`|`|s|q|l|␣|\n|S|E|L|E|C|T|␣|C|O|U|N|T|(|*|)|␣|F|R|O|M|␣|s|i|n|g|e|r|_|i|n|_|c|o|n|c|e|r|t|␣|\n|␣|␣|W]
-1.25:	[\n|`|`|`|s|q|l|␣|\n|S|E|L|E|C|T|␣|C|O|U|N|T|(|*|)|␣|F|R|O|M|␣|s|i|n|g|e|r|␣|I|N|␣|c|o|n|c|e|r|t|;|\n|`|`|`]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣|h]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣|s]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣|t]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|t|a|b|l|e|␣|a|l|i|a|s|e|s|␣|f|o|r|␣|s|t|

/teamspace/studios/this_studio/genlm-eval/genlm/eval/domains/spider/table_column_potential.py:88: UserWarning: Failed to parse complete 

# Create table aliases for stadium, singer, concert_name
  warnings.warn(f"Failed to parse complete {context}")


-1.28:	[\n|\n|#|␣|C|r|e|a|t|e|␣|t|a|b|l|e|␣|a|l|i|a|s|e|s|␣|f|o|r|␣|s|t|a|d|i|u|m|,|␣|s|i|n|g|e|r|,|␣|c|o|n|c|e|r|t|_|n|a|m|e|,]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣|h|o|w|␣|m|a|n]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣|t|h|e|␣|n|u|m]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|t|a|b|l|e|␣|a|l|i|a|s|e|s|␣|f|o|r|␣|s|t|a|d|i|u|m|,|␣|s|i|n|g|e|r|,|␣|c|o|n|c|e|r|t|␣|a|n|d|␣|s]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣|s|e|s|s|i|o|n|s]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣|h|o|w|␣|m|a|n|y]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|a|n|␣|a|g|g|r|e|g|a|t|e|d|␣|c|o|u|n|t|␣|o|f|␣|s|i|n|g|e|r|s|,|␣|c|o|u|n|t|i|n|g|␣|t|h|e|␣|n|u|m|b]
-0.79:	[\n|\n|#|␣|C|r|e|a|t|e|␣|t|a|b|l|e|␣|a|l|i|a|s|e|s|␣|f|o|r|␣|s|t|a|d|i|u|m|,|␣|s|i|n

In [ ]:
async def spider_model_adaptor(instance, output_dir: str, replicate: int) -> ModelOutput:
    global BYTE_LLM

    prompt_ids = default_prompt_formatter(
        BYTE_LLM.llm.tokenizer,
        instance,
        use_chat_format=False,
    )
    prompt_text = BYTE_LLM.llm.tokenizer.decode(prompt_ids)
    BYTE_LLM.set_prompt_from_str(prompt_text)

    # Print few-shot examples being used
    print("\n--- Few-shot examples ---")
    if instance.few_shot_examples:
        for i, (inp, out) in enumerate(instance.few_shot_examples):
            preview = inp.replace("\n", " ")
            if len(preview) > 200:
                preview = preview[:200] + "..."
            print(f"[{i}] Input: {preview}")
            print(f"    Output: {out}")
    else:
        print("(none)")

    # Print prompt and gold SQL for inspection
    print("\n--- Prompt ---\n" + prompt_text)
    print("\n--- Gold SQL ---\n" + instance.gold)
    condition = SpiderTableColumnVerifier(grammar=instance.lark_grammar, tables=instance.tables).coerce(BYTE_LLM, f=b"".join)
    sampler = AWRS(BYTE_LLM, condition)

    sequences = await sampler.smc(
        n_particles=5,
        ess_threshold=0.9,
        max_tokens=70,
        verbosity=1,
    )

    print("Decoded posterior:", sequences.decoded_posterior)

    responses: List[ModelResponse] = [
        ModelResponse(response=seq, weight=float(prob))
        for seq, prob in sequences.decoded_posterior.items()
    ]

    await BYTE_LLM.cleanup()

    return ModelOutput(responses=responses, runtime_seconds=None)